### Coding puzzles: Backtracking

[Playstore link for game](https://play.google.com/store/apps/details?id=zl.puzzle.riveriq&hl=en_IN)

Code the solution to these games (using recursion / backtracking)

In [18]:
class Dict:
    def __init__(self, d):
        self.dict = d

    # How to convert to string
    def __repr__(self):
        return str(self.dict)
    
    def __str__(self) -> str:
        return self.__repr__()
    
    def __eq__(self, other):
        return self.dict == other.dict
    
    def __hash__(self):
        return hash(
          str(list(sorted(self.dict.items())))
        )

d = Dict({"k": "v", "k2": "v2"})
d2 = Dict({"k2": "v2", "k": "v"})
set([d, d2])

{{'k': 'v', 'k2': 'v2'}}

In [21]:
set([1, 2, 4, False, "Hello", (1, 2, 3), 1.2322, 134343.12])

{(1, 2, 3), 1, 1.2322, 134343.12, 2, 4, False, 'Hello'}

In [24]:
class Entity:
  def __init__(self, name: str, time: int):
    self.name = name
    self.time = time
  
  def __repr__(self):
    return f'{self.name} ({self.time}s)'
  
  def __str__(self):
    return self.__repr__()
  
  def __eq__(self, other):
    if not isinstance(other, self.__class__):
      return False
    return self.name == other.name
  
  def __hash__(self):
    return hash(self.name)

a1 = Entity("Popeye", 12)
a2 = Entity("Popeye", 12)

a1 == a2
set([a1, a2])

{Popeye (12s)}

### Logic 3

Help a family of 5 people move across the river by boat, and the boat is capable of a maximum of 2 people carrying capacity. Time for travelling of each person in turn is 1s, 3s, 6s, 8s, and 12s. If two people both go on the boat, the boat will travel at the speed of the slower person. Find the minimum time required for the whole family to cross the river. (less than eual to 30s).

[Sample gamplay video](https://www.youtube.com/watch?v=HdcQjI2vSao&list=PLXO4k3Jc8d7d892be-P_H9J_vE_TN73qa&index=2)

In [25]:
"""
Help a family of 5 people move across the river by boat, and the boat is capable of a maximum of 2 people carrying capacity. Time for travelling of each person in turn is 1s, 3s, 6s, 8s, and 12s. If two people both go on the boat, the boat will travel at the speed of the slower person. Find the minimum time required for the whole family to cross the river. (less than eual to 30s).
"""

from typing import List, Set
from functools import total_ordering

@total_ordering
class Entity:
  """
  This class represents an entity in the river crossing problem. An entity is defined by the name of the person and the time he/she/they/it takes to cross the river.

  We override the __repr__ and __str__ methods of the class to provide a nice-looking string representation of the entity.
  A sample representation of the entity is "Popeye (3s)".

  We also need to make this class hashable to make sure we can insert an entity into the left and right sets of the state. To make the class hashable, we override the __eq__ and __hash__ methods.

  Attributes:
  name (str): The name of the person
  time (int): The time the person takes to cross the river
  """
  def __init__(self, name: str, time: int):
    self.name = name
    self.time = time
  
  def __repr__(self):
    return f'{self.name} ({self.time}s)'
  
  def __str__(self):
    return self.__repr__()
  
  # This method is required for comparison
  def __eq__(self, other):
    if not isinstance(other, self.__class__):
      return False
    return self.name == other.name
  
  # This method is required for hashing, and adding to set
  def __hash__(self):
    return hash(self.name)
  
  # This method is required for sorting.
  # Other similar methods like
  #  __gt__, __le__, __ge__ 
  # will be automatically implemented 
  # thanks to @total_ordering annotation
  def __lt__(self, other):
    return self.name < other.name


class State:
  """
  This class represents a state in the river crossing problem. A state is defined by the entities on the left side of the river, the entities on the right side of the river, the time left for the family to cross the river, and the direction of the next move.

  We override the __repr__ and __str__ methods of the class to provide a nice-looking string representation of the state.
  A sample representation of the state is
  "time: 30, left: (Popeye (3s), Olive (6s)), right: (Swee'Pea (1s), Wimpy (8s), Wotasnozzle (12s))".

  We also need to make this class hashable to make sure we can insert a state into the visited set. To make the class hashable, we override the __eq__ and __hash__ methods.

  Attributes:
  left (Set[Entity]): A set of entities on the left side of the river
  right (Set[Entity]): A set of entities on the right side of the river
  timeLeft (int): The time left for the family to cross the river
  nextLeft2right (bool): A boolean value that indicates the direction of the next move
  """
  def __init__(self, left: Set[Entity], right: Set[Entity], timeLeft: int, nextLeft2right: bool):
    self.left = left
    self.right = right
    self.timeLeft = timeLeft
    self.nextLeft2right = nextLeft2right
  
  def __repr__(self):
    return f'time: {self.timeLeft}, left: ({self.left}), right: ({self.right})'
  
  def __str__(self):
    return self.__repr__()
  
  def is_final(self):
    return len(self.left) == 0
  
  def is_valid(self, entity_count: int):
    """
    This method checks if the state is valid. A state is valid if the following conditions are met:
    1. The time left is greater than or equal to 0
    2. None of the entities are on both sides of the river
    3. The total number of entities on both sides of the river is equal to the provided total number of entities
    """
    if self.timeLeft < 0:
      return False
    if self.left & self.right:
      return False
    if len(self.left) + len(self.right) != entity_count:
      return False
    return True

  def __eq__(self, other):
    if not isinstance(other, self.__class__):
      return False
    return (
      self.left == other.left and
      self.right == other.right and
      self.timeLeft == other.timeLeft and
      self.nextLeft2right == other.nextLeft2right
    )
  
  def __hash__(self):
    """
    This method is required to make the class hashable. We use the hash of a tuple containing these elements to make the state hashable:
    1. A sorted string representation of the left entities
    2. A sorted string representation of the right entities
    3. The time left
    4. The direction of the next move
    """
    return hash((
      str(sorted(self.left)),
      str(sorted(self.right)),
      self.timeLeft,
      self.nextLeft2right
    ))


class Move:
  """
  This class represents a move in the river crossing problem. A move is defined by the direction of the move (left to right or right to left) and the entities that are moving in the move.

  We override the __repr__ and __str__ methods of the class to provide a nice-looking string representation of the move.
  A sample representation of the move is
  "left -> right: {Popeye (3s), Olive (6s)}".

  Attributes:
  isLeft2right (bool): A boolean value that indicates if the move is from left to right
  entities (Set[Entity]): A set of entities that are moving in the move
  """
  def __init__(self, isLeft2right: bool, entities: Set[Entity]):
    if not 0 < len(entities) <= 2:
      raise ValueError(f'Invalid number of people crossing the river: Number of people should be 1 or 2, but got {len(entities)}')
    self.isLeft2right = isLeft2right
    self.source = 'left' if isLeft2right else 'right'
    self.destination = 'right' if isLeft2right else 'left'
    self.entities = entities
  
  def __repr__(self):
    start = "left" if self.isLeft2right else "right"
    end = "right" if self.isLeft2right else "left"
    return f'{start} -> {end}: {self.entities}'

  def __str__(self):
    return self.__repr__()
  
  def validate(self, state: State):
    """
    This method validates if the move is valid.
    It does the following validations:
    1. Check if the move is in the correct direction according to the current state
    2. Check if all entities are in the source
    3. Check if none of the entities are in the destination

    Parameters:
    state (State): The current state

    Raises:
    ValueError: If the move is invalid

    Returns:
    None
    """
    source = state.left if self.isLeft2right else state.right
    destination = state.right if self.isLeft2right else state.left
    entities_not_in_source = self.entities - source
    entities_already_in_destination = self.entities & destination
    if self.isLeft2right != state.nextLeft2right:
      raise ValueError(f"""
        Invalid move: Expected {
        "left to right" if state.nextLeft2right else "right to left"
        } as next move, but got {
          "left to right" if self.isLeft2right else "right to left"}
        """)
    if entities_not_in_source:
      raise ValueError(f'Entities {entities_not_in_source} not in {self.source}. Current state: {state}')
    if entities_already_in_destination:
      raise ValueError(f'Entities {entities_already_in_destination} already in {self.destination}. Current state: {state}')
  
  def apply(self, state: State) -> State:
    """
    This method applies the move to the current state and returns the new state. The computation is done by copying the current state and applying the move to the copy.
    A copy is needed to avoid changing the same reference of the state object values.

    Parameters:
    state (State): The current state

    Returns:
    State: The new state after applying the move
    """
    # check if move is valid
    self.validate(state)
    # apply move
    left = state.left.copy()
    right = state.right.copy()
    if self.isLeft2right:
      left -= self.entities
      right |= self.entities
    else:
      left |= self.entities
      right -= self.entities
    timeLeft = state.timeLeft
    timeLeft -= max([entity.time for entity in self.entities])
    return State(left, right, timeLeft, not state.nextLeft2right)


class RiverCrossing:
  def __init__(self):
    self.entities = {
      Entity("Swee'Pea", 1),
      Entity("Popeye", 3),
      Entity("Olive", 6),
      Entity("Wimpy", 8),
      Entity("Wotasnozzle", 12)
    }
    self.initial_state = State(self.entities, set(), 30, True)
    self.visited = set()

  def get_possible_moves(self, state: State):
    """
    This method generates all possible moves from the current state. It uses yield to return a generator that yields all possible moves. We can also create a full list of moves and return it at once, but using a generator is more memory-efficient.
    
    Parameters:
    state (State): The current state

    Returns:
    Generator[Move]: A generator that yields all possible moves
    """
    for entity in state.left:
      yield Move(True, {entity})
      for entity1 in state.left:
        if entity == entity1:
          continue
        yield Move(True, {entity, entity1})
    for entity in state.right:
      yield Move(False, {entity})
      for entity1 in state.right:
        if entity == entity1:
          continue
        yield Move(False, {entity, entity1})

  def solve(self, state: State, moves: List[Move]):
    """
    This method solves the river crossing problem using backtracking.

    Parameters:
    state (State): The current state
    moves (List[Move]): The moves taken so far to reach the current state

    Returns:
    Optional[List[Move]]: The moves taken to reach the final state, or None if no solution is found
    """
    # print(state, moves)
    self.visited.add(state)
    if state.is_final():
      return moves
    for move in self.get_possible_moves(state):
      try:
        new_state = move.apply(state)
      except ValueError:
        continue
      if new_state and \
        new_state.is_valid(entity_count=len(self.entities)):
        if new_state in self.visited:
          continue
        result = self.solve(new_state, moves + [move])
        if result:
          return result
    return None

  def run(self):
    moves = self.solve(self.initial_state, [])
    if moves:
      print("Solution found:")
      for move in moves:
        print(move)
    else:
      print('No solution found')

RiverCrossing().run()

Solution found:
left -> right: {Popeye (3s), Swee'Pea (1s)}
right -> left: {Popeye (3s)}
left -> right: {Wimpy (8s), Wotasnozzle (12s)}
right -> left: {Swee'Pea (1s)}
left -> right: {Popeye (3s), Swee'Pea (1s)}
right -> left: {Swee'Pea (1s)}
left -> right: {Olive (6s), Swee'Pea (1s)}


### Logic 5

There are 6 frogs and 7 pillars, swap these 6 frogs into two groups of 3, provided that the frogs can not jump back, the longest distance they can jump is 2 pillars.

[Sample gameplay](https://www.youtube.com/watch?v=0HuJdtP5pHY&list=PLXO4k3Jc8d7d892be-P_H9J_vE_TN73qa&index=5)

In [30]:
from typing import List, Tuple

class RenderFrog:
  frogRight = """
  <svg fill="#000000" width="30px" height="30px" viewBox="-57.6 -57.6 691.20 691.20" xmlns="http://www.w3.org/2000/svg" transform="matrix(1, 0, 0, 1, 0, 0)" stroke="#000000" stroke-width="1.152"><g id="SVGRepo_bgCarrier" stroke-width="0"></g><g id="SVGRepo_tracerCarrier" stroke-linecap="round" stroke-linejoin="round"></g><g id="SVGRepo_iconCarrier"><path d="M446.53 97.43C439.67 60.23 407.19 32 368 32c-39.23 0-71.72 28.29-78.54 65.54C126.75 112.96-.5 250.12 0 416.98.11 451.9 29.08 480 64 480h304c8.84 0 16-7.16 16-16 0-17.67-14.33-32-32-32h-79.49l35.8-48.33c24.14-36.23 10.35-88.28-33.71-106.6-23.89-9.93-51.55-4.65-72.24 10.88l-32.76 24.59c-7.06 5.31-17.09 3.91-22.41-3.19-5.3-7.08-3.88-17.11 3.19-22.41l34.78-26.09c36.84-27.66 88.28-27.62 125.13 0 10.87 8.15 45.87 39.06 40.8 93.21L469.62 480H560c8.84 0 16-7.16 16-16 0-17.67-14.33-32-32-32h-53.63l-98.52-104.68 154.44-86.65A58.16 58.16 0 0 0 576 189.94c0-21.4-11.72-40.95-30.48-51.23-40.56-22.22-98.99-41.28-98.99-41.28zM368 136c-13.26 0-24-10.75-24-24 0-13.26 10.74-24 24-24 13.25 0 24 10.74 24 24 0 13.25-10.75 24-24 24z"></path></g></svg>
  """
  frogLeft = """
  <svg fill="#000000" width="30px" height="30px" viewBox="-57.6 -57.6 691.20 691.20" xmlns="http://www.w3.org/2000/svg" transform="matrix(-1, 0, 0, 1, 0, 0)" stroke="#000000" stroke-width="1.152"><g id="SVGRepo_bgCarrier" stroke-width="0"></g><g id="SVGRepo_tracerCarrier" stroke-linecap="round" stroke-linejoin="round"></g><g id="SVGRepo_iconCarrier"><path d="M446.53 97.43C439.67 60.23 407.19 32 368 32c-39.23 0-71.72 28.29-78.54 65.54C126.75 112.96-.5 250.12 0 416.98.11 451.9 29.08 480 64 480h304c8.84 0 16-7.16 16-16 0-17.67-14.33-32-32-32h-79.49l35.8-48.33c24.14-36.23 10.35-88.28-33.71-106.6-23.89-9.93-51.55-4.65-72.24 10.88l-32.76 24.59c-7.06 5.31-17.09 3.91-22.41-3.19-5.3-7.08-3.88-17.11 3.19-22.41l34.78-26.09c36.84-27.66 88.28-27.62 125.13 0 10.87 8.15 45.87 39.06 40.8 93.21L469.62 480H560c8.84 0 16-7.16 16-16 0-17.67-14.33-32-32-32h-53.63l-98.52-104.68 154.44-86.65A58.16 58.16 0 0 0 576 189.94c0-21.4-11.72-40.95-30.48-51.23-40.56-22.22-98.99-41.28-98.99-41.28zM368 136c-13.26 0-24-10.75-24-24 0-13.26 10.74-24 24-24 13.25 0 24 10.74 24 24 0 13.25-10.75 24-24 24z"></path></g></svg>
  """
  emptyBox = """
  <svg fill="#000000" width="30px" height="30px" viewBox="-57.6 -57.6 691.20 691.20" xmlns="http://www.w3.org/2000/svg" transform="matrix(-1, 0, 0, 1, 0, 0)" stroke="#000000" stroke-width="1.152"><g id="SVGRepo_bgCarrier" stroke-width="0"></g><g id="SVGRepo_tracerCarrier" stroke-linecap="round" stroke-linejoin="round"></g><g id="SVGRepo_iconCarrier"></g></svg>
  """

  def __init__(self, initialState: List[int]):
    self.state = initialState

  def _getHTMLForState(self, state: List[int]):
    html = ""
    for s in state:
      if s == 1:
        html += self.frogRight
      elif s == -1:
        html += self.frogLeft
      else:
        html += self.emptyBox
    return f"<div>{html}</div>"

  def html(self, moves: List[Tuple[int, int]]):
    h = f"""
    <style>
    svg {{
      padding: 5px 0;
    }}
    </style>
    """
    h += self._getHTMLForState(self.state)
    for move in moves:
      self.state[move[0]], self.state[move[1]] = \
        self.state[move[1]], self.state[move[0]]
      h += self._getHTMLForState(self.state)

    return h


In [31]:
class Frogs:
  def __init__(self):
    self.initial_state = [1, 1, 1, 0, -1, -1, -1]
    self.final_state = [-1, -1, -1, 0, 1, 1, 1]
    self.visited = set()
  
  def get_possible_moves(self, state):
    # get the position of the empty cell
    idx0 = state.index(0)
    # get the possible next states
    for idx1 in range(max(0, idx0 - 2), min(len(state), idx0 + 3)):
      # cell with 0, skip
      if state[idx1] == 0:
        continue
      # if the frog in the cell is facing the opposite direction of empty cell, skip
      if (state[idx1]) * (idx0 - idx1) < 0:
        continue
      # create a new state
      new_state = state.copy()
      new_state[idx0], new_state[idx1] = new_state[idx1], new_state[idx0]
      yield ((idx1, idx0), new_state)

  def solve(self, state, moves):
    self.visited.add(tuple(state))
    if state == self.final_state:
      return moves
    for move, new_state in self.get_possible_moves(state):
      if tuple(new_state) in self.visited:
        continue
      result = self.solve(new_state, moves + [move])
      if result:
        return result
    return None
  
  def run(self):
    moves = self.solve(self.initial_state, [])
    if not moves:
      print('No solution found')
      return
    return moves

moves = Frogs().run()
from IPython.core.display import HTML
h = RenderFrog([1, 1, 1, 0, -1, -1, -1]).html(moves)

# save html to file
with open(".temp/frogs.html", "w") as f:
  f.write(h)

HTML(h)